In [1]:
import numpy as np
from matplotlib import pyplot as plt
from matplotlib.patches import Rectangle
import mpl_toolkits.mplot3d.art3d as art3d
import py_gauge_mc

In [32]:
def p_to_dim_dim(p):
    if p == 3:
        return 1,2,3
    if p == 4:
        return 1,3,2
    if p == 5:
        return 2,3,1
    return None


def plot_state(state):
    T = np.arange(state.shape[0])
    X = np.arange(state.shape[1])
    Y = np.arange(state.shape[2])
    Z = np.arange(state.shape[3])
    P = np.arange(state.shape[4])
    TT, XX, YY, ZZ, PP = np.meshgrid(T,X,Y,Z,P)
    indices = np.stack([TT,XX,YY,ZZ,PP], axis=-1)
    nonzero_indices = indices[state!=0,:]
    
    fig = plt.figure()
    ax = plt.axes(projection='3d')
    ax.set_xlim((0,state.shape[1]))
    ax.set_ylim((0,state.shape[2]))
    ax.set_zlim((0,state.shape[3]))
    ax.set_xlabel('x')
    ax.set_ylabel('y')
    ax.set_zlabel('z')
    
    dims = ['x', 'y', 'z']
    for (t,x,y,z,p) in indices.reshape(-1,5):
        if t != 0:
            continue
        index = [x,y,z]
        
        pdims = p_to_dim_dim(p)
        
        if pdims is not None:
            first,second,normal = pdims
            
            zdir = dims[normal - 1]
            if zdir == "y":
                xy1, z1 = (x, z), y
            elif zdir == "x":
                xy1, z1 = (y, z), x
            else:
                xy1, z1 = (x, y), z
            
            p = Rectangle(xy1, 1, 1, color='blue', alpha=0.1 * abs(state[t,x,y,z,p]))
            ax.add_patch(p)
            art3d.pathpatch_2d_to_3d(p, z=z1, zdir=zdir)

In [40]:
graph = py_gauge_mc.GaugeTheory(8,8,8,8,[(n**2)*2 for n in range(30)])

for x in range(2,6):
    for y in range(2,6):
        graph.add_flux(0,x,y,3,3,amount=5)
        
for i in range(128):
    plot_state(graph.get_graph_state())
    plt.savefig('images/{}.png'.format(str(i).zfill(2)))
    plt.close()
    graph.run_local_update()

plot_state(graph.get_graph_state())
plt.savefig('images/{}.png'.format(str(i+1).zfill(2)))
plt.close()